In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


# ChatGPT Python API and Stock Selection Prompting Results

In [ ]:
import openai
import os
import pandas as pd
import time
import json
import csv
import pandas as pd

csv_file_path = "/content/drive/My Drive/MIE479 Capstone Project/Portfolio Construction/IWB_holdings.csv"
df = pd.read_csv(csv_file_path)
column_name = "Ticker"  # Replace with the actual column name
column_list = df[column_name].tolist()
#print(column_list)
openai.api_key = 'YOUR-API-KEY'

def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=0,
    )
    return response.choices[0].message["content"]
stock_frequency = {}

for i in range(0, 30):
    prompt = "Using a range of investing principles taken from leading funds, " \
         "create a theoretical fund comprising of at least 15 stocks (mention only their ticker name and nothing else) " \
         "from the Russell 1000 with the goal to outperform the Russell 1000, please give me a list of python's structure, only " \
         "give me the list and don't say any word else, thank you!"
    response = get_completion(prompt)
    #print(response)
    #response = '["AAPL", "MSFT", "AMZN", "FB", "GOOGL", "JNJ", "V", "PG", "JPM", "UNH", "MA", "INTC", "VZ", "HD", "T"]'
    stock_list = json.loads(response)

    for j in range(0, 15):
        count = 1
        if stock_list[j] not in stock_frequency and stock_list[j] in column_list:
            stock_frequency[stock_list[j]] = count
        elif stock_list[j] in stock_frequency:
            stock_frequency[stock_list[j]] += 1
print(stock_frequency)
stock_result_list = []
for stock in stock_frequency:
    stock_result_list.append(stock)
print(stock_result_list)

# Extract Assets Price from Yahoo Finance

In [ ]:
!pip install yfinance #if yahoo_fin is not installed

In [ ]:
import yfinance as yf
import pandas as pd
# Define the stock symbol and date range
stocks = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'JNJ', 'V', 'PG', 'JPM', 'UNH', 'MA', 'INTC', 'VZ', 'GOOG', 'HD', 'T']  # Update to ChatGPT output
SVM_stocks = ['TRGP', 'AMC', 'EXAS', 'OXY', 'LYV', 'JLL', 'FTI', 'OKE', 'SON', 'RRC', 'DVN', 'COTY', 'AR', 'EQT', 'NOV']
Clustering_stocks = ['AMC', 'AME', 'CRL', 'DVN', 'DFS', 'DTE', 'FERG', 'GIS', 'HD', 'MSFT', 'PRU', 'RUN', 'TMO', 'RARE', 'VIRT']

print(len(SVM_stocks))
print(len(Clustering_stocks))
start_date = "2016-07-01"
end_date = "2023-09-01"

# Fetch data and extract the adj close stock price
data = yf.download(Clustering_stocks, start=start_date, end=end_date)
df = data['Adj Close']
df_monthly = df.resample('MS').first()
print(df_monthly)

# Save the data as csv file
file_path = '/content/drive/My Drive/MIE479 Capstone Project/Portfolio Construction/Cluster_MIE479_AssetPrices.csv'
df_monthly.to_csv(file_path)

15
15
[*********************100%%**********************]  15 of 15 completed
                   AMC         AME         CRL         DFS         DTE  \
Date                                                                     
2016-07-01  274.720825   44.185452   83.300003   45.307018   66.680382   
2016-08-01  277.352142   44.832985   88.440002   47.543270   65.941589   
2016-09-01  297.525726   46.451855   83.599998   50.563496   61.931892   
2016-10-01  314.201233   45.251293   83.680000   48.026825   62.879299   
2016-11-01  299.378571   42.827957   74.080002   47.814877   63.861378   
...                ...         ...         ...         ...         ...   
2023-04-01   51.099998  143.772842  200.259995   96.585457  107.105530   
2023-05-01   56.500000  138.879089  190.100006   97.357597  111.006897   
2023-06-01   45.500000  146.463913  196.339996  103.660355  105.405449   
2023-07-01   44.400002  160.544907  208.669998  116.218079  110.256737   
2023-08-01   49.299999  159.187149 

#Extract Factors Data

In [ ]:
csv_file_path1 = "/content/drive/My Drive/MIE479 Capstone Project/Portfolio Construction/5_Factors_Monthly.csv"
df1 = pd.read_csv(csv_file_path1,skiprows=3,index_col=0)

selected = df1.loc['201608':'202308']
selected.iloc[:, 0:] = selected.iloc[:, 0:].apply(pd.to_numeric, errors='coerce')
selected.iloc[:, 0:] = selected.iloc[:, 0:] / 100

selected.index = pd.to_datetime(selected.index, format='%Y%m')
selected.index = selected.index.strftime('%Y-%m-%d')

selected = selected.rename_axis('Date', axis=1)
print(selected)

file_path = '/content/drive/My Drive/MIE479 Capstone Project/Portfolio Construction/MIE479_5FactorReturns.csv'
selected.to_csv(file_path, index_label='Date')

<ipython-input-10-f96edeaf81ca>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected.iloc[:, 0:] = selected.iloc[:, 0:].apply(pd.to_numeric, errors='coerce')
<ipython-input-10-f96edeaf81ca>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected.iloc[:, 0:] = selected.iloc[:, 0:] / 100


Date        Mkt-RF     SMB     HML     RMW     CMA      RF
2016-08-01  0.0049  0.0168  0.0318 -0.0185 -0.0034  0.0002
2016-09-01  0.0025  0.0185 -0.0124 -0.0221  0.0003  0.0002
2016-10-01 -0.0202 -0.0402  0.0409  0.0093  0.0028  0.0002
2016-11-01  0.0486  0.0707  0.0821 -0.0018   0.037  0.0001
2016-12-01  0.0181  0.0041  0.0353  0.0122 -0.0023  0.0003
...            ...     ...     ...     ...     ...     ...
2023-04-01  0.0061 -0.0256 -0.0004  0.0242  0.0286  0.0035
2023-05-01  0.0035 -0.0038 -0.0772 -0.0181 -0.0722  0.0036
2023-06-01  0.0646  0.0134 -0.0026  0.0218 -0.0162   0.004
2023-07-01  0.0321  0.0286  0.0411 -0.0057  0.0057  0.0045
2023-08-01 -0.0239 -0.0365 -0.0106  0.0343 -0.0237  0.0045

[85 rows x 6 columns]
